# FM-223: Lista 6

Aluno: Vinícius Freitas de Almeida

# Exercício 1

Resolva o exercício T4.11 da p. 180, sobre o cálculo de boxdim dos inteiros de 1
a 100 e dos racionais em [0,1]. Você acha que o conjunto de números racionais em
[0,1] é fractal? Por quê?

## Parte 1: Inteiros de 1 a 100

Primeiro, vamos calcular o boxdim dos inteiros de 1 a 100. Para isso, basta considerar a fórmula:

$$d = \lim_{\varepsilon \to 0^+} \frac{\log N(\varepsilon)}{\log 1/\varepsilon}$$

Em que $N(\varepsilon)$ é o número de caixas de lado $\varepsilon$ necessárias para cobrir o conjunto. Para calcular $N(\varepsilon)$, basta perceber que precisamos de apenas 100 caixas de tamanho $\varepsilon$ para cobrir o conjunto, pois cada inteiro é um ponto. Assim, $N(\varepsilon) = 100$ e:

$$d = \lim_{\varepsilon \to 0^+} \frac{\log 100}{\log 1/\varepsilon} = \lim_{\varepsilon \to 0^+} \frac{\log 100}{-\log \varepsilon} = 0

## Parte 2: Racionais em [0,1]

Considerando, agora, o conjunto $S = \mathbb{Q} \cap [0,1$], precisamos calcular $N(\varepsilon)$ para cada $\varepsilon$. Perceba que, se entre as caixas(intervalos) de comprimento $\varepsilon$ exista alguma lacuna de tamanho $\delta > 0$, então não é possível cobrir $S$ com essas caixas. Portanto, todas as caixas devem ser intervalos imediatamente subsequentes (e.g. $[0, \varepsilon], [\varepsilon, 2\varepsilon], \dots$). Assim, $N(\varepsilon) = 1/\varepsilon + O(1)$ e, portanto:

$$d = \lim_{\varepsilon \to 0^+} \frac{\log ( 1/\varepsilon + O(1))}{\log 1/\varepsilon} = 1$$


Com tal resultado, chega-se à conclusão de que o conjunto dos racionais não é um fractal, uma vez que ter dimensão não-inteira é condição necessária para ser um fractal.